<a href="https://colab.research.google.com/github/senchfu/repository1/blob/master/competition_bert_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 https://github.com/bojone/bert_in_keras/blob/master/sentiment.py

In [1]:
!ls

chinese_L-12_H-768_A-12      stopwords.txt     Train_DataSet.csv
chinese_L-12_H-768_A-12.zip  Test_DataSet.csv  Train_DataSet_Label.csv
sample_data		     Test.zip	       Train.zip


In [3]:
from google.colab import files
uploaded = files.upload()

Saving stopwords.txt to stopwords.txt
Saving Test.zip to Test.zip
Saving Train.zip to Train.zip


In [4]:
!unzip Train.zip && unzip Test.zip && ls

Archive:  Train.zip
  inflating: Train_DataSet.csv       
  inflating: Train_DataSet_Label.csv  
Archive:  Test.zip
  inflating: Test_DataSet.csv        
sample_data    Test_DataSet.csv  Train_DataSet.csv	  Train.zip
stopwords.txt  Test.zip		 Train_DataSet_Label.csv


In [0]:
import pandas as pd
import numpy as np

In [3]:
df_train_x = pd.read_csv('./Train_DataSet.csv')
df_train_x.head()

,id,title,content
0,7a3dd79f90ee419da87190cff60f7a86,问责领导(上黄镇党委书记张涛，宣国才真能一手遮天吗？),这几天看了有人举报施某某的贴子，经与举报人联系证实，是宣某当天中午请举报人和枪手喝酒后，晚上...
1,7640a5589bc7486ca199eeeb38af79dd,江歌事件:教会孩子，善良的同时更要懂得保护自己!,过去一年的江歌悲剧，这几日再次刷屏：住在东京都中野区的中国女留学生江歌，收留了被前男友陈世锋...
2,8c5bda93e4ba401f90a0faa5b28fe57f,"绝味鸭脖广告""开黄腔""引众怒 ""双11""这么拼值吗?",“双11”1600亿的销售额让中国乃至全世界感到震惊，为此，不仅有不惜欠债百万的“剁手党”，...
3,1aa777fed31a4b8a9d866f05b5477557,央视曝光!如东一医药企业将槽罐车改成垃圾车，夜间偷排高浓度废水,2016年以来，如东高新区一家叫做拜瑞生物医药的企业竟然将装废水的槽罐车伪装成洒水车，常年在...
4,6c67ac55360340258e157f3710ebae6c,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发了!,新闻资讯•民生热点•城市生活•南通人都在看浏览更多资讯请关注微信公众号:南通传媒网昨晚央视一...


In [4]:
df_train_y = pd.read_csv('./Train_DataSet_Label.csv')
print(df_train_y['label'].unique())

[2 1 0]


In [5]:
df_train_x['id'].values == df_train_y['id'].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [6]:
id_x = df_train_x['id']
id_x.head()

0    7a3dd79f90ee419da87190cff60f7a86
1    7640a5589bc7486ca199eeeb38af79dd
2    8c5bda93e4ba401f90a0faa5b28fe57f
3    1aa777fed31a4b8a9d866f05b5477557
4    6c67ac55360340258e157f3710ebae6c
Name: id, dtype: object

In [0]:
labels = []
for idd in id_x:
  if df_train_y[df_train_y['id']==idd].empty:
    labels.append(-1)
  else:
    labels.append(df_train_y[df_train_y['id']==idd]['label'].values.item())
    #np.append(labels, df_train_y[df_train_y['id']==idd]['label'].values, 0)

In [8]:
len(labels)

7345

In [9]:
labels[:10]

[2, 1, 2, 2, 2, 2, 2, 1, 2, 2]

In [0]:
for x in labels:
  if not isinstance(x, int):
    print(x)

In [0]:
df_train_x['label'] = labels

In [12]:
df_train_x.head()

,id,title,content,label
0,7a3dd79f90ee419da87190cff60f7a86,问责领导(上黄镇党委书记张涛，宣国才真能一手遮天吗？),这几天看了有人举报施某某的贴子，经与举报人联系证实，是宣某当天中午请举报人和枪手喝酒后，晚上...,2
1,7640a5589bc7486ca199eeeb38af79dd,江歌事件:教会孩子，善良的同时更要懂得保护自己!,过去一年的江歌悲剧，这几日再次刷屏：住在东京都中野区的中国女留学生江歌，收留了被前男友陈世锋...,1
2,8c5bda93e4ba401f90a0faa5b28fe57f,"绝味鸭脖广告""开黄腔""引众怒 ""双11""这么拼值吗?",“双11”1600亿的销售额让中国乃至全世界感到震惊，为此，不仅有不惜欠债百万的“剁手党”，...,2
3,1aa777fed31a4b8a9d866f05b5477557,央视曝光!如东一医药企业将槽罐车改成垃圾车，夜间偷排高浓度废水,2016年以来，如东高新区一家叫做拜瑞生物医药的企业竟然将装废水的槽罐车伪装成洒水车，常年在...,2
4,6c67ac55360340258e157f3710ebae6c,恶劣至极，央视都曝光了!南通如东一医药企业将槽罐车改成洒水车，夜间偷排高浓度废水...丢大发了!,新闻资讯•民生热点•城市生活•南通人都在看浏览更多资讯请关注微信公众号:南通传媒网昨晚央视一...,2


In [13]:
df_train_x = df_train_x.dropna()
df_train_x.shape

(7265, 4)

In [14]:
df_train_x[df_train_x['label']==-1].shape

(0, 4)

In [15]:
df_test_x = pd.read_csv('./Test_DataSet.csv')
print(df_test_x.shape)
df_test_x.head()

(7356, 3)


,id,title,content
0,00005a3efe934a19adc0b69b05faeae7,九江办好人民满意教育,近3年来，九江市紧紧围绕“人本教育、公平教育、优质教育、幸福教育”的目标，努力办好人民满意教...
1,0009dc82c41341d794837271d4dbff5f,中央第三生态环境保护督察组转办我市第三十一批信访件办理情况,中央第三生态环境保护督察组转办我市的第三十一批信访件共计1件，截至12月11日，已全部办结，...
2,000f3763b6074588817c9ab90a22a814,大雨天车被淹，保险公司该不该赔？,核心提示：近日，连续的降雨天气造成中心城区部分路段积水严重，一些市民驾车出行涉水时造成车辆被...
3,00117934dbe240068c95b6d04b08eea7,英特尔新cpu微架构ocean cove曝光,intel在主流平台上确认，今年将推出基于14nm的whiskeylake，明年开始大规模出...
4,0014cef5ccfa43b3a6b20162a03763fa,公安部侦破一批重大网络赌博案件 德州约局平台成重灾区,2018年4月，公安部指挥河南、北京、广西等地公安机关联合行动，成功侦破北京联众公司棋牌事业...


In [16]:
df_test_x = df_test_x.dropna()
df_test_x.shape

(7288, 3)

In [0]:
def remove_blankspace(our_data):
    m1 = map(lambda s: s.replace(' ', ''), our_data)
    return list(m1)

In [0]:
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False

def format_str(content):
    content_str = ''
    for i in content:
        if is_chinese(i):
            content_str = content_str + ｉ
    return content_str

In [0]:
def man_data(data):
  list_title_content = list(data['content'].values)
  list_title_content = remove_blankspace(list_title_content)
  chinese_list = []
  for line in list_title_content:
    chinese_list.append(format_str(line))
  return chinese_list

In [0]:
datas = [df_train_x, df_test_x]
clean_title = []
for data in datas:
  clean_title.append(man_data(data))

In [0]:
datas = [df_train_x, df_test_x]
clean_content = []
for data in datas:
  clean_content.append(man_data(data))

In [23]:
train_x_content = clean_content[0]
train_x_content[:2]

['这几天看了有人举报施某某的贴子经与举报人联系证实是宣某当天中午请举报人和枪手喝酒后晚上才发的贴子本人不去讨论前二天的举报相信总归会有说法的今天一看施全军年月日实名举报上黄镇宣国才的贴子仍被锁定禁止评论已经正好一整年了图片施全军实名举报天后上黄镇党委政府回复如下图图片图片一年的贴子再次被网友顶起来后才发现施某几天前回复网友的处理结果竟如下图图片现责问张涛书记宣国才被举报这么多问题什么时候有答复宣国才被举报后为什么被立刻免了村书记职务为什么又被安排到城管队吃空响自己却天天在我们水泥厂上班赚黑钱这几个月水泥每吨近元纯利润还供不应求宣国才还清上黄政府担保借给宣国才代付振东厂工资社保的钱了吗据了解宣国才占他人企业经营又欠税万元欠社保万元应该还欠了职工工资几十万上黄政府打算替宣国才担保还是归还我们厂合法会计和老板被判刑四到六年现在服刑厂子给宣国才强占宣国才每天赚多万净利润却对外宣称天天亏本等咱老板刑满回厂宣国才给咱厂天天亏可能要亏的几千万元甚至几个亿张涛书记您承担还是上黄政府承担当初可是您亲自把厂交给宣国才生产的希望徐市长看到本贴后能像批示批示违建等民生问题一样关注一下我们水泥厂的将来也请徐市长抽日理万机之空亲自约谈一下当事人特别是那位施站长千万不能听取一面之辞',
 '过去一年的江歌悲剧这几日再次刷屏住在东京都中野区的中国女留学生江歌收留了被前男友陈世锋恶意纠缠的闺蜜刘鑫两人在回到江歌公寓楼时陈世锋已经等在楼下叫嚣着要刘鑫给自己一个说法男友此时的情绪处于濒临崩溃的状态江歌为了保护刘鑫就让她先进了房间自己拦在外面要求陈世锋离开结果江歌被陈世锋用刀多处刺伤脖子和胸部刀刀毙命残忍至极最终因失血过多丧生十几刀刘鑫躲在屋里躲在门后亲耳听着闺蜜江歌的声声求助及惨叫却始终没有打开门连邻居都听到呼救纷纷开门查看究竟那扇可以救命的们始终没有打开江歌死后刘鑫面对警方的询问称自己一无所知什么都没有听见不肯出来指证凶手为江歌伸冤只想着撇清关系澄清自己任江歌遭外界议论指摘甚至拒绝同江歌妈妈联系自己做了新头发买了新包包快快乐乐地过着自己的生活暗自庆幸自己终于摆脱了那个纠缠自己的前男友迫于舆论的压力刘鑫终于出现了江歌为刘鑫失去了生命可在刘鑫眼里江歌的一条命抵不过自己的名声她只在意自己及家人的生活受到了严重的影响认为自己知道错了江歌妈妈就该原谅她了不应该揪着她不放甚至在事搁一年后首次与她死者母亲见面还身

In [24]:
clean_content[0][:2]

['这几天看了有人举报施某某的贴子经与举报人联系证实是宣某当天中午请举报人和枪手喝酒后晚上才发的贴子本人不去讨论前二天的举报相信总归会有说法的今天一看施全军年月日实名举报上黄镇宣国才的贴子仍被锁定禁止评论已经正好一整年了图片施全军实名举报天后上黄镇党委政府回复如下图图片图片一年的贴子再次被网友顶起来后才发现施某几天前回复网友的处理结果竟如下图图片现责问张涛书记宣国才被举报这么多问题什么时候有答复宣国才被举报后为什么被立刻免了村书记职务为什么又被安排到城管队吃空响自己却天天在我们水泥厂上班赚黑钱这几个月水泥每吨近元纯利润还供不应求宣国才还清上黄政府担保借给宣国才代付振东厂工资社保的钱了吗据了解宣国才占他人企业经营又欠税万元欠社保万元应该还欠了职工工资几十万上黄政府打算替宣国才担保还是归还我们厂合法会计和老板被判刑四到六年现在服刑厂子给宣国才强占宣国才每天赚多万净利润却对外宣称天天亏本等咱老板刑满回厂宣国才给咱厂天天亏可能要亏的几千万元甚至几个亿张涛书记您承担还是上黄政府承担当初可是您亲自把厂交给宣国才生产的希望徐市长看到本贴后能像批示批示违建等民生问题一样关注一下我们水泥厂的将来也请徐市长抽日理万机之空亲自约谈一下当事人特别是那位施站长千万不能听取一面之辞',
 '过去一年的江歌悲剧这几日再次刷屏住在东京都中野区的中国女留学生江歌收留了被前男友陈世锋恶意纠缠的闺蜜刘鑫两人在回到江歌公寓楼时陈世锋已经等在楼下叫嚣着要刘鑫给自己一个说法男友此时的情绪处于濒临崩溃的状态江歌为了保护刘鑫就让她先进了房间自己拦在外面要求陈世锋离开结果江歌被陈世锋用刀多处刺伤脖子和胸部刀刀毙命残忍至极最终因失血过多丧生十几刀刘鑫躲在屋里躲在门后亲耳听着闺蜜江歌的声声求助及惨叫却始终没有打开门连邻居都听到呼救纷纷开门查看究竟那扇可以救命的们始终没有打开江歌死后刘鑫面对警方的询问称自己一无所知什么都没有听见不肯出来指证凶手为江歌伸冤只想着撇清关系澄清自己任江歌遭外界议论指摘甚至拒绝同江歌妈妈联系自己做了新头发买了新包包快快乐乐地过着自己的生活暗自庆幸自己终于摆脱了那个纠缠自己的前男友迫于舆论的压力刘鑫终于出现了江歌为刘鑫失去了生命可在刘鑫眼里江歌的一条命抵不过自己的名声她只在意自己及家人的生活受到了严重的影响认为自己知道错了江歌妈妈就该原谅她了不应该揪着她不放甚至在事搁一年后首次与她死者母亲见面还身

In [25]:
train_x_title = clean_title[0]
test_x = clean_title[1]
train_y = df_train_x['label'].values
train_y = [y.item() for y in train_y]
train_y[:10]

[2, 1, 2, 2, 2, 2, 2, 1, 2, 2]

In [26]:
train_x_content[:2]

['这几天看了有人举报施某某的贴子经与举报人联系证实是宣某当天中午请举报人和枪手喝酒后晚上才发的贴子本人不去讨论前二天的举报相信总归会有说法的今天一看施全军年月日实名举报上黄镇宣国才的贴子仍被锁定禁止评论已经正好一整年了图片施全军实名举报天后上黄镇党委政府回复如下图图片图片一年的贴子再次被网友顶起来后才发现施某几天前回复网友的处理结果竟如下图图片现责问张涛书记宣国才被举报这么多问题什么时候有答复宣国才被举报后为什么被立刻免了村书记职务为什么又被安排到城管队吃空响自己却天天在我们水泥厂上班赚黑钱这几个月水泥每吨近元纯利润还供不应求宣国才还清上黄政府担保借给宣国才代付振东厂工资社保的钱了吗据了解宣国才占他人企业经营又欠税万元欠社保万元应该还欠了职工工资几十万上黄政府打算替宣国才担保还是归还我们厂合法会计和老板被判刑四到六年现在服刑厂子给宣国才强占宣国才每天赚多万净利润却对外宣称天天亏本等咱老板刑满回厂宣国才给咱厂天天亏可能要亏的几千万元甚至几个亿张涛书记您承担还是上黄政府承担当初可是您亲自把厂交给宣国才生产的希望徐市长看到本贴后能像批示批示违建等民生问题一样关注一下我们水泥厂的将来也请徐市长抽日理万机之空亲自约谈一下当事人特别是那位施站长千万不能听取一面之辞',
 '过去一年的江歌悲剧这几日再次刷屏住在东京都中野区的中国女留学生江歌收留了被前男友陈世锋恶意纠缠的闺蜜刘鑫两人在回到江歌公寓楼时陈世锋已经等在楼下叫嚣着要刘鑫给自己一个说法男友此时的情绪处于濒临崩溃的状态江歌为了保护刘鑫就让她先进了房间自己拦在外面要求陈世锋离开结果江歌被陈世锋用刀多处刺伤脖子和胸部刀刀毙命残忍至极最终因失血过多丧生十几刀刘鑫躲在屋里躲在门后亲耳听着闺蜜江歌的声声求助及惨叫却始终没有打开门连邻居都听到呼救纷纷开门查看究竟那扇可以救命的们始终没有打开江歌死后刘鑫面对警方的询问称自己一无所知什么都没有听见不肯出来指证凶手为江歌伸冤只想着撇清关系澄清自己任江歌遭外界议论指摘甚至拒绝同江歌妈妈联系自己做了新头发买了新包包快快乐乐地过着自己的生活暗自庆幸自己终于摆脱了那个纠缠自己的前男友迫于舆论的压力刘鑫终于出现了江歌为刘鑫失去了生命可在刘鑫眼里江歌的一条命抵不过自己的名声她只在意自己及家人的生活受到了严重的影响认为自己知道错了江歌妈妈就该原谅她了不应该揪着她不放甚至在事搁一年后首次与她死者母亲见面还身

In [27]:
train_x_title[:2]

['问责领导上黄镇党委书记张涛宣国才真能一手遮天吗', '江歌事件教会孩子善良的同时更要懂得保护自己']

In [0]:
import jieba
def fenci(datas):
    cut_words = map(lambda s: list(jieba.cut(s)), datas)
    return list(cut_words)

In [0]:
train_x_content = fenci(train_x_content)
train_x_content[:2]  #分词后的content

In [0]:
stopwords = []
with open('stopwords.txt', 'r') as f:
  for line in f.readlines():
    stopwords.append(line.replace('\n', ''))
stopwords[:30]

In [0]:
def drop_stopwords(contents, stopwords):
    contents_clean = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
        contents_clean.append(line_clean)
    return contents_clean

In [32]:
train_x_content = drop_stopwords(train_x_content, stopwords)
print(train_x_content[:2])

[['几天', '有人', '举报', '施', '贴子', '举报人', '联系', '证实', '宣某', '当天', '中午', '请', '举报人', '枪手', '喝酒', '晚上', '发', '贴子', '不去', '讨论', '前', '二天', '举报', '相信', '总归', '会', '说法', '今天', '看施', '全军', '年月日', '实名', '举报', '黄镇', '宣国', '贴子', '锁定', '禁止', '评论', '已经', '正好', '整年', '图片', '施', '全军', '实名', '举报', '天后', '黄镇', '党委政府', '回复', '图', '图片', '图片', '一年', '贴子', '再次', '网友', '顶', '起来', '发现', '施某', '几天', '前', '回复', '网友', '处理结果', '竟', '图', '图片', '现', '责问', '张涛', '书记', '宣国', '举报', '问题', '答复', '宣国', '举报', '立刻', '免', '村', '书记', '职务', '安排', '城管', '队', '吃', '空响', '天天', '水泥厂', '上班', '赚', '黑钱', '几个', '月', '水泥', '每吨', '近元', '纯利润', '供不应求', '宣国', '还清', '黄', '政府', '担保', '借给', '宣国', '代付', '振', '东厂', '工资', '社保', '钱', '了解', '宣国', '占', '企业', '经营', '欠税', '万元', '欠', '社保', '万元', '应该', '欠', '职工工资', '几十万', '黄', '政府', '打算', '宣国', '担保', '归还', '厂', '合法', '会计', '老板', '判刑', '四到', '六年', '现在', '服刑', '厂子', '宣国', '强占', '宣国', '每天', '赚', '多万', '净利润', '对外', '宣称', '天天', '亏本', '老板', '刑满', '回厂', '宣国', '咱厂', '天天', '亏', '可能', '亏', '几千万元', '几个', '亿张', '涛

In [33]:
train_x_content_str = []
for content in train_x_content:
  train_x_content_str.append(''.join(content))
print(train_x_content_str[:2])

['几天有人举报施贴子举报人联系证实宣某当天中午请举报人枪手喝酒晚上发贴子不去讨论前二天举报相信总归会说法今天看施全军年月日实名举报黄镇宣国贴子锁定禁止评论已经正好整年图片施全军实名举报天后黄镇党委政府回复图图片图片一年贴子再次网友顶起来发现施某几天前回复网友处理结果竟图图片现责问张涛书记宣国举报问题答复宣国举报立刻免村书记职务安排城管队吃空响天天水泥厂上班赚黑钱几个月水泥每吨近元纯利润供不应求宣国还清黄政府担保借给宣国代付振东厂工资社保钱了解宣国占企业经营欠税万元欠社保万元应该欠职工工资几十万黄政府打算宣国担保归还厂合法会计老板判刑四到六年现在服刑厂子宣国强占宣国每天赚多万净利润对外宣称天天亏本老板刑满回厂宣国咱厂天天亏可能亏几千万元几个亿张涛书记承担黄政府承担当初亲自厂交给宣国生产希望徐市长看到本贴批示批示违建民生问题关注一下水泥厂将来请徐市长抽日理万机之空亲自约谈一下当事人特别那位施站长千万不能听取一面之辞', '过去一年江歌悲剧几日再次刷屏住东京都中野区中国女留学生江歌收留前男友陈世锋恶意纠缠闺蜜刘鑫两人回到江歌公寓楼时陈世锋已经楼下叫嚣刘鑫一个说法男友情绪处于濒临崩溃状态江歌保护刘鑫就让先进房间拦在外面要求陈世锋离开江歌陈世锋刀多处刺伤脖子胸部刀刀毙命残忍至极最终失血过多丧生十几刀刘鑫躲屋里躲门亲耳听闺蜜江歌声声求助惨叫始终没有打开门邻居听到呼救纷纷开门查看究竟那扇救命始终没有打开江歌死刘鑫面对警方询问称一无所知没有听见不肯指证凶手为江歌伸冤想着撇清关系澄清任江歌遭外界议论指摘拒绝同江歌妈妈联系做新头发买新包包快快乐乐地过生活暗自庆幸终于摆脱纠缠前男友迫于舆论压力刘鑫终于出现江歌刘鑫失去生命刘鑫眼里江歌一条命抵不过名声在意家人生活严重影响认为知道错江歌妈妈原谅应该揪不放事搁一年首次死者母亲见面身穿艳色裤子摘帽子丝毫没有尊重江歌遇害一次出现网络那头争辩利益作斗争如果说江歌正直善良江歌妈妈言传身教刘鑫自私冷漠刘鑫妈妈江歌妈妈争执中可见一斑刘鑫妈妈说命短闺女江歌妈妈后悔一定后悔后悔教女儿善良付出生命代价过错买单曾经一个事件一位爸爸带孩子河边钓鱼遇到一位带孩子玩妈妈相识两个孩子很快打成一片玩得不亦乐乎突然两个孩子掉进河里这位爸爸二话没说立刻下河救孩子孩子岸边近一个孩子已经飘得比较远孩子妈妈不会游泳一直岸边喊话救救孩子这位爸爸稍犹豫奋力孩子游过去孩子拖岸边这才一个孩

In [34]:
len(test_x)

7288

In [31]:
a = [1,2,3]
b = [4,5,6]
for i, j in zip(a, b):
  print(i,j)

1 4
2 5
3 6


In [35]:
!pip install -q keras-bert keras-rectified-adam
!wget -q https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip -o chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [36]:
import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs

Using TensorFlow backend.


In [0]:
maxlen = 100
config_path = 'chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'chinese_L-12_H-768_A-12/vocab.txt'

In [0]:
token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


In [0]:
class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

In [58]:
a = [1,2,3]
b = [4,5,6]
c = [7,8,9]
for i,j,k in zip(zip(a, b), c):
  print(i,j,k)

ValueError: ignored

In [0]:
data = []

for content, label in zip(train_x_title, train_y):
    data.append((content, label))
for content, label in zip(train_x_content_str, train_y):
    data.append((content, label))

In [41]:
data[:5]

[('问责领导上黄镇党委书记张涛宣国才真能一手遮天吗', 2),
 ('江歌事件教会孩子善良的同时更要懂得保护自己', 1),
 ('绝味鸭脖广告开黄腔引众怒双这么拼值吗', 2),
 ('央视曝光如东一医药企业将槽罐车改成垃圾车夜间偷排高浓度废水', 2),
 ('恶劣至极央视都曝光了南通如东一医药企业将槽罐车改成洒水车夜间偷排高浓度废水丢大发了', 2)]

In [42]:
data[-2]

('日日照经济技术开发区发布奎山汽贸城火灾事故调查情况报告年月日时分日照经济技术开发区奎山汽贸城日照贵华汽车销售服务有限公司发生火灾过火面积约平方米直接损失约万元火灾未造成人员伤亡事故发生工委管委主要领导第一时间赶赴事故现场指挥部署抢险救援事故调查处理工作国务院办公厅消防安全责任制实施办法安全生产法消防法生产安全事故报告调查处理条例法律法规工委管委成立管委分管领导任组长纪工委公安分局经发局安监局综合行政执法局消防大队奎山街道分管负责人成员事故调查领导小组全面开展事故调查处理工作事故调查组四不放过科学严谨依法依规实事求是注重实效原则现场勘验查阅资料调查取证委托技术鉴定机构事故现场相关物证设备设施开展取样分析技术鉴定查明事故发生原因认定事故性质责任事故暴露出问题提出防范措施现将情况报告基本情况事发建筑基本情况事发建筑位置事发建筑位于日照经济技术开发区南昌路西侧郭家湖子桥南约米奎山汽贸城事发建筑未取得公安机关规定编制门牌楼地址事发建筑布局事发建筑钢结构彩钢板顶东西长约米南北宽约米占地面积平方米建筑总面积约平方米事发建筑分为展厅维修车间两部分地上部分一层局部二层整体呈凹字形结构事发建筑共有单位家均取得营业执照北侧日照贵华汽车销售服务有限公司中间日照凌峰工贸有限公司南侧日照市凌青汽车销售服务有限公司事发建筑系集销售维修仓库办公一体建筑起火建筑日照贵华汽车销售服务有限公司位于事发建筑北侧分为展厅办公区维修车间起火建筑设有出入口西侧出入口进入展厅西北侧出入口进入展厅北侧墙体中间出入口维修车间客户休息区进入办公区北侧维修车间出入口进入维修车间二事故相关单位情况日照贵华汽车销售服务有限公司法定代表人为周萌萌实际控制人周传运统一社会信用代码类型有限责任公司自然人投资控股住所山东省日照经济开发区奎山汽贸城南昌路以东包头路以北经营范围汽车汽车配件销售二类机动车维修汽车装饰日照凌峰工贸有限公司法定代表人为凌吉峰统一社会信用代码类型有限责任公司自然人投资控股住所日照经济技术开发区奎山汽贸城经营范围汽车销售二类机动车维修车辆信息咨询服务汽车装饰汽车配件建材销售日照市凌青汽车销售服务有限公司法定代表人庄须青统一社会信用代码类型有限责任公司自然人投资住所山东省日照经济开发区奎山汽贸城国道包头路交汇处北米经营范围汽车销售二类机动车维修车辆信息咨询服务二手车销售汽车装饰汽车配件安装销售汽车租赁电子产品销

In [0]:
# 按照9:1的比例划分训练集和验证集
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]

In [44]:
print(len(train_data))
print(len(valid_data))

13077
1453


In [59]:
#train_data = data
print(len(train_data))
print(len(valid_data))

13077
1453


In [0]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []


In [0]:
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

In [0]:
bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

In [0]:
x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

In [0]:
x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(3, activation='softmax')(x)

In [71]:
model = Model([x1_in, x2_in], p)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_7 (Model)                 (None, None, 768)    101677056   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 768)          0           model_7[1][0]              

In [0]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)


In [73]:
model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=5,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

Epoch 1/5
409/409 [==============================] - 305s 746ms/step - loss: 0.5687 - acc: 0.7647 - val_loss: 0.4494 - val_acc: 0.8156
Epoch 2/5
409/409 [==============================] - 294s 720ms/step - loss: 0.4116 - acc: 0.8327 - val_loss: 0.4365 - val_acc: 0.8273
Epoch 3/5
409/409 [==============================] - 294s 720ms/step - loss: 0.2834 - acc: 0.8897 - val_loss: 0.4758 - val_acc: 0.8142
Epoch 4/5
409/409 [==============================] - 295s 720ms/step - loss: 0.1715 - acc: 0.9351 - val_loss: 0.5724 - val_acc: 0.8183
Epoch 5/5
  7/409 [..............................] - ETA: 4:40 - loss: 0.0819 - acc: 0.9732

KeyboardInterrupt: ignored

In [41]:
!nvidia-smi

Sun Sep  1 13:59:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    74W / 149W |   8344MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
def predict_sentiment(text):
    tokens = tokenizer.tokenize(text)
    indices = np.array([[token_dict[token] for token in tokens]])
    segments = np.zeros(len(tokens))
    result = model.predict([indices, segments], verbose=False)[0].argmax(-1)
    return result

In [0]:
results = []
for sent in test_x:
  results.append(predict_sentiment(sent))

In [44]:
len(results)

7288

In [45]:
results[:20]

[0, 1, 1, 1, 2, 0, 0, 1, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 1, 2]

In [0]:
pd.DataFrame({"id": df_test_x['id'].values, "label": results}).to_csv('results_bert_v4.csv', index=False, header=True)

In [0]:
from google.colab import files
files.download('results_bert_v4.csv')